<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

# Define the function to generate a random architecture
def random_architecture(input_dim, output_dim):
    layers = []
    for _ in range(random.randint(1, 3)):  # Randomly select between 1 and 3 layers
        layers.append(nn.Linear(input_dim, random.randint(4, 16)))  # Random layer size
        layers.append(nn.ReLU())  # ReLU activation
        input_dim = layers[-2].out_features  # Update input_dim for the next layer
    layers.append(nn.Linear(input_dim, output_dim))  # Output layer
    return nn.Sequential(*layers)  # Create the model

# Define the function to train and evaluate the model
def train_and_evaluate(model, data, labels, epochs=5):
    criterion = nn.CrossEntropyLoss()  # Loss function
    optimizer = optim.Adam(model.parameters(), lr=0.01)  # Optimizer
    for _ in range(epochs):
        optimizer.zero_grad()  # Zero the gradients
        output = model(data)  # Forward pass
        loss = criterion(output, labels)  # Compute the loss
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update the model parameters
    return loss.item()  # Return the final loss

# Example NAS loop
data = torch.randn(100, 4)  # Example input data (batch_size=100, input_dim=4)
labels = torch.randint(0, 2, (100,))  # Example labels (batch_size=100, num_classes=2)

best_model = None
best_loss = float('inf')
for _ in range(10):  # Perform NAS for 10 iterations
    model = random_architecture(input_dim=4, output_dim=2)  # Generate random architecture
    loss = train_and_evaluate(model, data, labels)  # Train and evaluate the model
    if loss < best_loss:  # Check if the current model is better
        best_loss = loss
        best_model = model

print("Best model loss:", best_loss)  # Print the best model loss